In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("Resources/vine_table.csv")

In [3]:
df_20 = df[df.total_votes > 20]
#df_20

In [4]:
df_helpful = df_20[df.helpful_votes / df.total_votes >= 0.5]
df_helpful

,review_id,star_rating,helpful_votes,total_votes,vine,verified_purchase
0,R31EUR60FV3BU5,4,32,39,N,N
2,R3MDX24QDAT0OW,4,35,37,N,N
3,R2YVFZB4ESDT7D,5,139,141,N,Y
5,R2R8EWH1JUZCSI,5,16,21,N,N
7,R3I8GORXGEJL3R,4,95,103,N,Y
...,...,...,...,...,...,...
210428,R2AUIIZ3MK6UG5,5,69,82,N,N
210429,R2KJ3U2FUMVUEA,2,36,42,N,N
210430,R2EPRX3Y4R54ET,4,19,22,N,N
210432,R3KYIA96PD8P8V,5,34,34,N,N


In [5]:
# data frame for vine = y 
df_vine_y = df_helpful[df_helpful.vine =='Y']
#df_vine_y


In [6]:
# data frame for vine = n
df_vine_n = df_helpful[df_helpful.vine =='N']

In [7]:
yes_5_star_df = df_vine_y['star_rating'].value_counts().to_frame().sort_index(ascending = False)
yes_5_star_df

,star_rating
5,8
4,12
3,16
2,5
1,3


In [8]:
yes_perc_of_total = df_vine_y.groupby(['star_rating']).count()['total_votes'].to_frame().rename(columns={"star_rating": "star_rating_y","total_votes":"pct of total yes votes"})

yes_perc_of_total = yes_perc_of_total.apply(lambda x: x/x.sum()).sort_index(ascending = False)

In [9]:
#yes_perc_of_total = pd.Series(["{0:.2f}%".format(val) for val in yes_perc_of_total['pct of total_yes_votes']],index = yes_perc_of_total.index)
yes_perc_of_total

,pct of total yes votes
star_rating,
5,0.181818
4,0.272727
3,0.363636
2,0.113636
1,0.068182


In [10]:
y_result = pd.concat([yes_5_star_df,yes_perc_of_total],axis=1,join="inner").sort_index(ascending = False)
y_result = y_result.rename(columns={'star_rating':'star rating y'})

In [11]:
no_5_star_df = df_vine_n['star_rating'].value_counts().to_frame()
no_5_star_df
#state_office = no_5_star_df.groupby(['star_rating'])
#no_5_star_perc = no_5_star_df.groupby(level=0).apply(lambda x: x * 100 / float(x.sum()))
#no_5_star_perc

,star_rating
5,73276
4,23698
1,23006
3,12890
2,8772


In [12]:
no_perc_of_total = df_vine_n.groupby(['star_rating']).count()['total_votes'].to_frame().rename(columns={"star_rating": "star_rating n","total_votes":"pct of total no votes"})

no_perc_of_total = no_perc_of_total.apply(lambda x: round(x/x.sum(),2)).sort_index(ascending = False)

no_perc_of_total

,pct of total no votes
star_rating,
5,0.52
4,0.17
3,0.09
2,0.06
1,0.16


In [13]:
n_result = pd.concat([no_5_star_df,no_perc_of_total],axis=1,join="inner").sort_index(ascending = False)
n_result = n_result.rename(columns={'star_rating':'star rating n'})

In [14]:
result = pd.concat([y_result,n_result],axis = 1,join="inner")
result

,star rating y,pct of total yes votes,star rating n,pct of total no votes
5,8,0.181818,73276,0.52
4,12,0.272727,23698,0.17
3,16,0.363636,12890,0.09
2,5,0.113636,8772,0.06
1,3,0.068182,23006,0.16


In [15]:
result.loc['total'] = result.select_dtypes(pd.np.number).sum()
result

<ipython-input-15-d1992de2981a>:1: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  result.loc['total'] = result.select_dtypes(pd.np.number).sum()


,star rating y,pct of total yes votes,star rating n,pct of total no votes
5,8.0,0.181818,73276.0,0.52
4,12.0,0.272727,23698.0,0.17
3,16.0,0.363636,12890.0,0.09
2,5.0,0.113636,8772.0,0.06
1,3.0,0.068182,23006.0,0.16
total,44.0,1.000000,141642.0,1.00


In [16]:
result.style.format({
    'star rating y':'{:,.0f}'.format,
    'pct of total yes votes':'{:,.2%}'.format,
    'star rating n':'{:,.0f}'.format,
    'pct of total no votes':'{:,.2%}'.format,
})

,star rating y,pct of total yes votes,star rating n,pct of total no votes
5,8,18.18%,"73,276",52.00%
4,12,27.27%,"23,698",17.00%
3,16,36.36%,"12,890",9.00%
2,5,11.36%,"8,772",6.00%
1,3,6.82%,"23,006",16.00%
total,44,100.00%,"141,642",100.00%


In [17]:
result.columns = pd.MultiIndex.from_product([['In Vine Program','Not in Vine Program'],['star_rating','pct of total votes']])
result

#df.columns = pd.MultiIndex.from_product([[1,2],['A','B']])

In Vine Program                    Not in Vine Program  \
          star_rating pct of total votes         star_rating   
5                 8.0           0.181818             73276.0   
4                12.0           0.272727             23698.0   
3                16.0           0.363636             12890.0   
2                 5.0           0.113636              8772.0   
1                 3.0           0.068182             23006.0   
total            44.0           1.000000            141642.0   

                          
      pct of total votes  
5                   0.52  
4                   0.17  
3                   0.09  
2                   0.06  
1                   0.16  
total               1.00

In [18]:
result.style.format('{:,.0f}',subset=[col for col in result.columns if col[1] == "star_rating"]).format('{:,.2%}',subset=[col for col in result.columns if col[1] == "pct of total votes"])




   
    
